# Patterns Visualization
Similar to other notebooks, this one focuses on visualizing the patterns observed in the target patterns, which contains information about Human retina and relates that to P6 mouse cortex.

In [ ]:
import random
random.seed(a=613)
import numpy as np
import scProject
import scanpy as sc

patterns = sc.read('data/hretinaPatterns.csv')
dataset = sc.read_h5ad('data/test_target.h5ad')
dataset.X[dataset.X<0] = 0
scProject.matcher.orthologMapper(dataset, 'data/gene_map.csv','Mouse gene stable ID','Gene stable ID', 'gene_id')
dataset_filtered, patterns_filtered = scProject.matcher.filterAnnDatas(dataset, patterns.transpose(), 'gene_id')

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']= [15, 20]
dataset_filtered = scProject.matcher.logTransform(dataset_filtered)
scProject.rg.NNLR_ElasticNet(dataset_filtered, patterns_filtered, 'GV01', .00001, .9, layer='log', iterations=1000000)
scProject.viz.pearsonMatrix(dataset_filtered, patterns_filtered, 'CellType', 12, 'GV01', 'GV01Pears', True, xtickSize=10, ytickSize=10, row_cluster=False, col_cluster=False)

In [ ]:
plt.rcParams['figure.figsize']= [12, 10]
scProject.viz.UMAP_Projection(dataset_filtered, 'CellType', 'GV01', 'UMAPMG01', 20)

In [ ]:
plt.rcParams['figure.figsize']= [12, 10]
scProject.viz.featurePlots(dataset_filtered, [4, 23], 'GV01', 'UMAPMG01', subset=['Excitatory Neurons'], obsColumn='CellType')

In [ ]:
scProject.viz.patternWeightDistribution(dataset_filtered, 'GV01', [4, 23], obsColumn='CellType', subset=['Excitatory Neurons'], numBins=100)

In [ ]:
glia= dataset_filtered[dataset_filtered.obs['CellType'].isin(['Excitatory Neurons'])]
# We are now going to construct two clusters of microglia based on feature expression of 5
C1 = glia[glia.obsm['GV01'][:, 22] > .3]
C2 = glia[glia.obsm['GV01'][:, 22] <= .3]

C1.X = np.log1p(C1.X) #log transform for statistical tests
C2.X = np.log1p(C2.X) #log transform for statistcal tests

# In order to do the regression we had to filter genes, but now we are going to use the whole feature map
AllGenes = dataset[dataset.obs['CellType'].isin(['Excitatory Neurons'])]
# transfer regression insights
AllGenes.obsm['GV01'] = glia.obsm['GV01']

cluster1 = AllGenes[glia.obsm['GV01'][:, 22] > .3]
cluster2 = AllGenes[glia.obsm['GV01'][:, 22] <=.3]

cluster1.X = np.log1p(cluster1.X) #log transform for stats
cluster2.X = np.log1p(cluster2.X)

print(cluster1.shape, cluster2.shape)

In [ ]:
scProject.stats.featureExpressionSig(C1, 'GV01', 4, .99999999999)

In [ ]:
scProject.stats.HotellingT2(C1, C2)

In [ ]:
plt.rcParams['figure.figsize']= [10,15]
df=scProject.stats.projectionDriver(patterns_filtered, C1, C2,.95,'gene_short_name', 96)
print(df[0])